<a href="https://colab.research.google.com/github/bhosmer/fold/blob/main/notebooks/fold_shapes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook

This is part of a series of notebooks that will give a high-level introduction to **fold**'s unified array model. It's part of the **fold** repo [here](https://github.com/bhosmer/fold). 

In this notebook we'll introduce **generalized shapes**. Later notebooks will move on to
* advanced indexing
* strided storage 
* ragged arrays
* views
* sparsity 


### Colab setup

In [1]:
!git clone https://github.com/bhosmer/fold.git
import sys
sys.path.insert(0,'/content/fold')

Cloning into 'fold'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 1), reused 12 (delta 0), pack-reused 5
Unpacking objects: 100% (20/20), done.


In [2]:
from fold import *

# Generalized shapes

There are two reasons a fully capable multidimensional array model must be able to express array shapes other than (hyper)rectangles (TIL: [orthotopes](https://en.wiktionary.org/wiki/orthotope)): 

1. most immediately, many use cases involve "ragged" data, which will otherwise need to be padded (or stored in multiple arrays).
2. more deeply, much of the internal infrastructure we seek to unify can be naturally expressed with nonrectangular arrays, avoiding a proliferation of bespoke data structures. 

Our goal is to _generalize_ the current shape abstraction, rather than replace it. Specifically, we'd like to satisfy the following constraints:

* rectangular dimensions should be expressed as they are in the current abstraction: a positive integer specifies the _extent_ of the dimension.
* every shape (rectangular or not) should be entirely described, as now, by a tuple of descriptors, one per dimension.
* all dimensions (rectangular or not) should be described using a common set of desciptive components
* array shapes should be free to mix rectangular and nonrectangular dimensions as needed, constrained only by generic well-formedness conditions across dimensions.

**fold**'s shape data model satisfies these constraints via the following reformulation:
1. each per-dimension descriptor in a shape tuple denotes not a unitary extent but a _sequence of extents_, one for each _position_ in that dimension.
2. sequences can be _encoded_, using descriptors which capture different kinds of internal regularity.

Thus rectangular dimensions are extremely regular (since they have the same extent at every position) and can be described with a _single value_. In the **fold** shape model, this value is interpreted as an encoding - a compressed representation - of a sequence. Other encodings are available to represent other kinds of sequences, up to and including fully enumerated lists of extents.

What follows is a survey of some array shapes expressible in **fold** - not an exhaustive catalog, but a handful of examples to build intuition and spur ideas. 

For now, the best way to dive deeper is to look at the [implementations](https://github.com/bhosmer/fold/blob/main/fold/dim.py) and [tests](https://github.com/bhosmer/fold/blob/main/test/test_dim.py).)

## Rectangular dimensions

Rectangular dimensions are the most regular of all - they're defined as having the same extent at every position. Accordingly, rectangular dimensions can be described as usual with single values, as in this 4-by-4 matrix:

In [3]:
# rand() generates a randomized array of a given shape and optional dtype
mat = rand(4, 4)
print(mat)
print()
print(mat.shape)

[[0.7607, 0.4060, 0.7831, 0.4426],
 [0.4571, 0.1512, 0.7671, 0.0574],
 [0.2272, 0.7540, 0.4080, 0.5127],
 [0.4221, 0.6482, 0.2364, 0.5511]]

(4, 4)


Aside: the shift in perspective from _extent_ to _extent sequence_ becomes more concrete if we look at **fold**'s internal representation, which caches dimension _lengths_:

In [4]:
print(repr(mat.shape))

Shape(dims=(Rect(w=4, n=1), Rect(w=4, n=4)))


We won't dwell on the implications here, but it's worth taking a moment to think about this interpretation, latent in all shape metadata. 

What `(4, 4)` actually tells us is:
* this array is (a single value) made up of 4 rows
* each of the 4 rows contains 4 columns.

The beauty of the standard shape model for rectangular arrays is that it completely elides the tree-flavored semantics implied by this interpretation, allowing shapes to be manipulated much more freely as descriptions of _spaces_ made up of orthogonal dimensions.

Nonrectangularity forces a certain amount of tree-like character back to the surface, but in the form of coherence constraints between dimensions, rather than explicit nesting in the shape model. This is key. 

(End of aside.)

## Ragged dimensions

At the other extreme, _entirely_ ragged dimensions (i.e., dimensions whose extents have no internal regularity) are "incompressible" and must be enumerated. 

Here we create a ragged 2D array with 8 rows of varying width:

In [5]:
ragged = rand(8, [7, 11, 5, 6, 10, 3, 0, 1])
print(ragged)
print()
print(ragged.shape)

[[0.6499, 0.3736, 0.4192, 0.0375, 0.5094, 0.5547, 0.6376],
 [0.3319, 0.7465, 0.2317, 0.9845, 0.5598, 0.4236, 0.4585, 0.9895, 0.9825, 0.1505, 0.5073],
 [0.5372, 0.8105, 0.2062, 0.4819, 0.2162],
 [0.4667, 0.3428, 0.5437, 0.6038, 0.1505, 0.1989],
 [0.6116, 0.9520, 0.8587, 0.1512, 0.2081, 0.0498, 0.1088, 0.8528, 0.5375, 0.0720],
 [0.8279, 0.5920, 0.3185],
 [],
 [0.3127]]

(8, [7, 11, 5, 6, 10, 3, 0, 1])


Here the shift in perspective from unitary extents to extent _sequences_ becomes obvious. 

Aside: an important property of **fold**'s generalized shapes is that each entry in a shape tuple is a _complete linear description_ of that dimension: there is no nesting. For example, note the resulting shapes when we split our ragged 2D array into first even, then uneven batches:

In [6]:
print("even batches:")
ragged3d_even = ragged.reshape(4, 2, ragged.shape[-1])
print(ragged3d_even)
print()
print(ragged3d_even.shape)

print("\nuneven batches:")
ragged3d_uneven = ragged.reshape(4, [2, 1, 2, 3], ragged.shape[-1])
print(ragged3d_uneven)
print()
print(ragged3d_uneven.shape)

even batches:
[[[0.6499, 0.3736, 0.4192, 0.0375, 0.5094, 0.5547, 0.6376],
  [0.3319, 0.7465, 0.2317, 0.9845, 0.5598, 0.4236, 0.4585, 0.9895, 0.9825, 0.1505, 0.5073]],

 [[0.5372, 0.8105, 0.2062, 0.4819, 0.2162],
  [0.4667, 0.3428, 0.5437, 0.6038, 0.1505, 0.1989]],

 [[0.6116, 0.9520, 0.8587, 0.1512, 0.2081, 0.0498, 0.1088, 0.8528, 0.5375, 0.0720],
  [0.8279, 0.5920, 0.3185]],

 [[],
  [0.3127]]]

(4, 2, [7, 11, 5, 6, 10, 3, 0, 1])

uneven batches:
[[[0.6499, 0.3736, 0.4192, 0.0375, 0.5094, 0.5547, 0.6376],
  [0.3319, 0.7465, 0.2317, 0.9845, 0.5598, 0.4236, 0.4585, 0.9895, 0.9825, 0.1505, 0.5073]],

 [[0.5372, 0.8105, 0.2062, 0.4819, 0.2162]],

 [[0.4667, 0.3428, 0.5437, 0.6038, 0.1505, 0.1989],
  [0.6116, 0.9520, 0.8587, 0.1512, 0.2081, 0.0498, 0.1088, 0.8528, 0.5375, 0.0720]],

 [[0.8279, 0.5920, 0.3185],
  [],
  [0.3127]]]

(4, [2, 1, 2, 3], [7, 11, 5, 6, 10, 3, 0, 1])


## Capturing regularity

In between these two extremes, **fold** defines an assortment of descriptors that capture different kinds of regularity, including:

- a constant rate of change (i.e., affinity)
- repetition of extents and extent sequences
- sparsity
- composition

Here's a high-level overview of `Dim` ADT, as represented by their descriptors:
```
Dim ::= Seq | Rect | Affine | Repeat | Runs | Sparse | Chain
Seq ::= List[Nat]       // sequence of extents (uncompressed)
Rect ::= Nat | (Nat, Nat)    // extent or (extent, length) 
Affine ::= (Nat, Nat, Int)   // initial extent, length, step size
Repeat ::= (Dim, Nat)        // Dim, number of reps
Runs ::= (Dim, Dim)          // extents, reps per extent (equal length)
Sparse ::= Map[Nat, Nat]     // map from offsets to extents
Chain ::= List[Dim]          // sequence of Dims
```

A full exploration of fold's dimension data model is out of scope for this introduction, but here are a few more examples to build intuition and spur ideas. 

(For now, the best way to dive deeper is to look at the [implementations](https://github.com/bhosmer/fold/blob/main/fold/dim.py) and [tests](https://github.com/bhosmer/fold/blob/main/test/test_dim.py).)



## Affine dimensions

Where a **Rectangular** dimension is defined by a constant extent across its sequence of positions, an **Affine** dimension is defined by an initial extent which then changes at a constant _rate_ across its sequence of positions.

As shown in our examples so far, **fold** uses Python literals as dimension descriptors in factory APIs and prints, translating them to and from [strongly-typed internal representations](https://github.com/bhosmer/fold/blob/main/fold/dim.py). 

We've seen rectangular dimensions described by single integers, and ragged dimensions described by integer lists. **Affine** dimensions are described by int triples `(start, length, step)`. For example, 
* `(1, 5, 1)` denotes the sequence `[1, 2, 3, 4, 5]`
* `(5, 5, -1)` denotes the sequence `[5, 4, 3, 2, 1]`

A typical use case for Affine dimensions is in describing triangular shapes:

In [7]:
print("lower triangle:")
tril = rand(5, (1, 5, 1))
print()
print(tril)
print()
print(tril.shape)

print("\nupper triangle:")
triu = rand(5, (5, 5, -1))
print()
print(triu)
print()
print(triu.shape)

lower triangle:

[[0.5005],
 [0.0492, 0.3665],
 [0.1135, 0.8439, 0.3467],
 [0.1413, 0.2974, 0.4344, 0.6402],
 [0.7822, 0.3771, 0.0618, 0.7209, 0.3603]]

(5, (1, 5, 1))

upper triangle:

[[0.4887, 0.5401, 0.6735, 0.0336, 0.5833],
 [0.0423, 0.5661, 0.2924, 0.4563],
 [0.5096, 0.3490, 0.4387],
 [0.0048, 0.5848],
 [0.8848]]

(5, (5, 5, -1))


## Repetition

Dimension descriptors also can be combined, to capture compound regularity - for example, here we repeat a 2-dimensional lower triangle over a third dimension.

Note our use of _implicit repetition in shape construction_ here: a dimension given to a shape constructor is automatically repeated until it fills the space created by its outward dimensions (but the repetition must be an even multiple). Compare the inner dimension descriptor passed to `rand()` with the corresponding descriptor printed in `trils.shape`. 

In [8]:
print("stack of lower triangles:")
trils = rand(3, 5, (1, 5, 1))
print()
print(trils)
print()
print(trils.shape)

stack of lower triangles:

[[[0.6579],
  [0.2794, 0.6838],
  [0.4275, 0.2834, 0.1974],
  [0.3202, 0.9427, 0.4792, 0.5836],
  [0.5526, 0.3051, 0.7195, 0.0184, 0.4668]],

 [[0.4050],
  [0.4398, 0.6663],
  [0.6044, 0.6794, 0.6114],
  [0.6520, 0.3872, 0.4233, 0.5883],
  [0.1278, 0.4867, 0.0876, 0.1873, 0.7618]],

 [[0.2666],
  [0.5676, 0.5466],
  [0.2310, 0.3801, 0.9611],
  [0.5146, 0.3636, 0.4190, 0.0999],
  [0.3782, 0.0109, 0.2689, 0.3419, 0.8178]]]

(3, 5, ((1, 5, 1), 3))


The standard shape transforms also work* - for example here we use `reshape` to collapse the outer two dimensions of `trils` above, producing a 2-dimensional array whose inner dimension has internal repetition.

_*where they make sense - it's out of scope for this introduction, but some operations don't have canonical generalizations to all nonrectangular shapes, e.g. permute._

In [9]:
print("2D array with lower triangular fringe:")
flat_trils = trils.reshape(-1, (1, 5, 1))
print()
print(flat_trils)
print()
print(flat_trils.shape)

2D array with lower triangular fringe:

[[0.6579],
 [0.2794, 0.6838],
 [0.4275, 0.2834, 0.1974],
 [0.3202, 0.9427, 0.4792, 0.5836],
 [0.5526, 0.3051, 0.7195, 0.0184, 0.4668],
 [0.4050],
 [0.4398, 0.6663],
 [0.6044, 0.6794, 0.6114],
 [0.6520, 0.3872, 0.4233, 0.5883],
 [0.1278, 0.4867, 0.0876, 0.1873, 0.7618],
 [0.2666],
 [0.5676, 0.5466],
 [0.2310, 0.3801, 0.9611],
 [0.5146, 0.3636, 0.4190, 0.0999],
 [0.3782, 0.0109, 0.2689, 0.3419, 0.8178]]

(15, ((1, 5, 1), 3))


## Expansion

Here we expand one of our 2D triangles into a new rectangular inner dimension:

In [10]:
print("original 2d tril:")
print()
print(tril)
print()
print(tril.shape)

print("\n...expanded:")
tril3 = tril.unsqueeze(-1).expand(-1, -1, 5)
print()
print(tril3)
print()
print(tril3.shape)

original 2d tril:

[[0.5005],
 [0.0492, 0.3665],
 [0.1135, 0.8439, 0.3467],
 [0.1413, 0.2974, 0.4344, 0.6402],
 [0.7822, 0.3771, 0.0618, 0.7209, 0.3603]]

(5, (1, 5, 1))

...expanded:

[[[0.5005, 0.5005, 0.5005, 0.5005, 0.5005]],

 [[0.0492, 0.0492, 0.0492, 0.0492, 0.0492],
  [0.3665, 0.3665, 0.3665, 0.3665, 0.3665]],

 [[0.1135, 0.1135, 0.1135, 0.1135, 0.1135],
  [0.8439, 0.8439, 0.8439, 0.8439, 0.8439],
  [0.3467, 0.3467, 0.3467, 0.3467, 0.3467]],

 [[0.1413, 0.1413, 0.1413, 0.1413, 0.1413],
  [0.2974, 0.2974, 0.2974, 0.2974, 0.2974],
  [0.4344, 0.4344, 0.4344, 0.4344, 0.4344],
  [0.6402, 0.6402, 0.6402, 0.6402, 0.6402]],

 [[0.7822, 0.7822, 0.7822, 0.7822, 0.7822],
  [0.3771, 0.3771, 0.3771, 0.3771, 0.3771],
  [0.0618, 0.0618, 0.0618, 0.0618, 0.0618],
  [0.7209, 0.7209, 0.7209, 0.7209, 0.7209],
  [0.3603, 0.3603, 0.3603, 0.3603, 0.3603]]]

(5, (1, 5, 1), 5)


Here we instead expand into a nonrectangular inner dimension, defined using the `Runs` descriptor above:

In [11]:
print("\n...expanded into nonrectangular inner dimension:")

INNER = ((1, 5, 1), (1, 5, 1))
print(f"INNER descriptor {INNER}")
print(f"INNER expansion {[n for n in dim(INNER)]}")

tril3 = tril.unsqueeze(-1).expand(-1, -1, INNER)
print()
print(tril3)
print()
print(tril3.shape)


...expanded into nonrectangular inner dimension:
INNER descriptor ((1, 5, 1), (1, 5, 1))
INNER expansion [1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5]

[[[0.5005]],

 [[0.0492, 0.0492],
  [0.3665, 0.3665]],

 [[0.1135, 0.1135, 0.1135],
  [0.8439, 0.8439, 0.8439],
  [0.3467, 0.3467, 0.3467]],

 [[0.1413, 0.1413, 0.1413, 0.1413],
  [0.2974, 0.2974, 0.2974, 0.2974],
  [0.4344, 0.4344, 0.4344, 0.4344],
  [0.6402, 0.6402, 0.6402, 0.6402]],

 [[0.7822, 0.7822, 0.7822, 0.7822, 0.7822],
  [0.3771, 0.3771, 0.3771, 0.3771, 0.3771],
  [0.0618, 0.0618, 0.0618, 0.0618, 0.0618],
  [0.7209, 0.7209, 0.7209, 0.7209, 0.7209],
  [0.3603, 0.3603, 0.3603, 0.3603, 0.3603]]]

(5, (1, 5, 1), ((1, 5, 1), (1, 5, 1)))


## Other examples

**Common-width, equal-size batches**: each batch contains the same quantity of items; all items within a batch share a common width. 

Note our use of the `Runs` encoding described above.

In [12]:
NUM_BATCHES = 3
BATCH_SIZE = 4
BATCH_WIDTHS = [5, 7, 3]

SEQUENCE_LENGTHS = (BATCH_WIDTHS, (BATCH_SIZE, NUM_BATCHES))
print(f"SEQUENCE_LENGTHS descriptor {SEQUENCE_LENGTHS}")
print(f"SEQUENCE_LENGTHS internal {repr(dim(SEQUENCE_LENGTHS))}")
print(f"SEQUENCE_LENGTHS expansion {[n for n in dim(SEQUENCE_LENGTHS)]}")

t = rand(NUM_BATCHES, BATCH_SIZE, SEQUENCE_LENGTHS)
print()
print(t)
print()
print(t.shape)

SEQUENCE_LENGTHS descriptor ([5, 7, 3], (4, 3))
SEQUENCE_LENGTHS internal Runs(vals=Seq(offs=[0, 5, 12, 15]), reps=Rect(w=4, n=3))
SEQUENCE_LENGTHS expansion [5, 5, 5, 5, 7, 7, 7, 7, 3, 3, 3, 3]

[[[0.4139, 0.8632, 0.3629, 0.0277, 0.2755],
  [0.2566, 0.6736, 0.7702, 0.5568, 0.4502],
  [0.9248, 0.3641, 0.3990, 0.1835, 0.9283],
  [0.6119, 0.9373, 0.3817, 0.3536, 0.9559]],

 [[0.1551, 0.3135, 0.2931, 0.2317, 0.4130, 0.8142, 0.6612],
  [0.8336, 0.5592, 0.7605, 0.5804, 0.7483, 0.0095, 0.8250],
  [0.5170, 0.4770, 0.1149, 0.4263, 0.1730, 0.4689, 0.6499],
  [0.6944, 0.9344, 0.8651, 0.6143, 0.4508, 0.9232, 0.2636]],

 [[0.2530, 0.8622, 0.2153],
  [0.3563, 0.2013, 0.9023],
  [0.7885, 0.6264, 0.7460],
  [0.3812, 0.2806, 0.7643]]]

(3, 4, ([5, 7, 3], (4, 3)))


**Common-width, unequal-size batches**: batches contain different quantities of items; all items within a batch share a common width, again making use of `Runs`.

In [13]:
NUM_BATCHES = 3
BATCH_SIZES = [5, 3, 4]
BATCH_WIDTHS = [5, 7, 3]

SEQUENCE_LENGTHS = (BATCH_WIDTHS, BATCH_SIZES)
print(f"SEQUENCE_LENGTHS descriptor {SEQUENCE_LENGTHS}")
print(f"SEQUENCE_LENGTHS internal {repr(dim(SEQUENCE_LENGTHS))}")
print(f"SEQUENCE_LENGTHS expansion {[n for n in dim(SEQUENCE_LENGTHS)]}")

t = rand(NUM_BATCHES, BATCH_SIZES, SEQUENCE_LENGTHS)
print()
print(t)
print()
print(t.shape)

SEQUENCE_LENGTHS descriptor ([5, 7, 3], [5, 3, 4])
SEQUENCE_LENGTHS internal Runs(vals=Seq(offs=[0, 5, 12, 15]), reps=Seq(offs=[0, 5, 8, 12]))
SEQUENCE_LENGTHS expansion [5, 5, 5, 5, 5, 7, 7, 7, 3, 3, 3, 3]

[[[0.4542, 0.2290, 0.1632, 0.9855, 0.9956],
  [0.0803, 0.6060, 0.3532, 0.4492, 0.5224],
  [0.3214, 0.2364, 0.7230, 0.3295, 0.9729],
  [0.1219, 0.5274, 0.7844, 0.3037, 0.8317],
  [0.2985, 0.2986, 0.0088, 0.1138, 0.8097]],

 [[0.5940, 0.4876, 0.1535, 0.8721, 0.9498, 0.0196, 0.8803],
  [0.2752, 0.7718, 0.9046, 0.2694, 0.0978, 0.5218, 0.0906],
  [0.9402, 0.1004, 0.1986, 0.2537, 0.6221, 0.4900, 0.5233]],

 [[0.2901, 0.2333, 0.1007],
  [0.3480, 0.0518, 0.3684],
  [0.6850, 0.0165, 0.9710],
  [0.6694, 0.4620, 0.7515]]]

(3, [5, 3, 4], ([5, 7, 3], [5, 3, 4]))
